# Plasma Beta with FGM and FPI data

This notebook shows how to calculate plasma beta with FGM and FPI data

## Get started

In [1]:
from pyspedas.mms import fgm, fpi # load routines
from pyspedas import tinterpol # interpolate tplot variables
from pytplot import tplot, get_data, store_data # plot, access, and create tplot variables

## Define some constants

In [2]:
mu0 = 1256.0 # nT-m/A
Kb = 1.3807*10**(-16.) # cm^2-g-1/s^2-1/K

## Set some options for the load routines

In [3]:
probe = '1'
trange = ['2015-10-16/11:00', '2015-10-16/14:00']

## Load the FGM and FPI data

In [4]:
fgm_vars = fgm(trange=trange, probe=probe)
fpi_vars = fpi(datatype=['dis-moms', 'des-moms'], trange=trange, probe=probe, center_measurement=True)

18-Oct-19 00:22:35: Loading /Volumes/data/data/mms/mms1/fgm/srvy/l2/2015/10/mms1_fgm_srvy_l2_20151016_v4.18.0.cdf


The lengths of x and y do not match!
Loaded variables:
mms1_fgm_b_gse_srvy_l2
mms1_fgm_b_gsm_srvy_l2
mms1_fgm_b_dmpa_srvy_l2
mms1_fgm_b_bcs_srvy_l2
mms1_fgm_flag_srvy_l2
mms1_fgm_r_gse_srvy_l2
mms1_fgm_r_gsm_srvy_l2
mms1_fgm_hirange_srvy_l2
mms1_fgm_bdeltahalf_srvy_l2
mms1_fgm_stemp_srvy_l2
mms1_fgm_etemp_srvy_l2
mms1_fgm_mode_srvy_l2
mms1_fgm_rdeltahalf_srvy_l2


18-Oct-19 00:23:08: Loading /Volumes/data/data/mms/mms1/fpi/fast/l2/dis-moms/2015/10/mms1_fpi_fast_l2_dis-moms_20151016100000_v3.3.0.cdf
18-Oct-19 00:23:08: Loading /Volumes/data/data/mms/mms1/fpi/fast/l2/dis-moms/2015/10/mms1_fpi_fast_l2_dis-moms_20151016120000_v3.3.0.cdf
18-Oct-19 00:23:08: Loading /Volumes/data/data/mms/mms1/fpi/fast/l2/des-moms/2015/10/mms1_fpi_fast_l2_des-moms_20151016100000_v3.3.0.cdf
18-Oct-19 00:23:08: Loading /Volumes/data/data/mms/mms1/fpi/fast/l2/des-moms/2015/10/mms1_fpi_fast_l2_des-moms_20151016120000_v3.3.0.cdf


Cannot find x axis.
No attribute named DEPEND_TIME or DEPEND_0 in                           variable mms1_des_compressionloss_fast
Cannot find x axis.
No attribute named DEPEND_TIME or DEPEND_0 in                           variable mms1_des_compressionloss_fast
Cannot find x axis.
No attribute named DEPEND_TIME or DEPEND_0 in                           variable mms1_dis_compressionloss_fast
Cannot find x axis.
No attribute named DEPEND_TIME or DEPEND_0 in                           variable mms1_dis_compressionloss_fast
Loaded variables:
mms1_des_errorflags_fast
mms1_des_startdelphi_count_fast
mms1_des_startdelphi_angle_fast
mms1_des_pitchangdist_lowen_fast
mms1_des_pitchangdist_miden_fast
mms1_des_pitchangdist_highen_fast
mms1_des_energyspectr_px_fast
mms1_des_energyspectr_mx_fast
mms1_des_energyspectr_py_fast
mms1_des_energyspectr_my_fast
mms1_des_energyspectr_pz_fast
mms1_des_energyspectr_mz_fast
mms1_des_energyspectr_par_fast
mms1_des_energyspectr_anti_fast
mms1_des_energyspectr_perp

## Set some variable names

In [5]:
temp_para_i = 'mms'+probe+'_dis_temppara_fast'
temp_perp_i = 'mms'+probe+'_dis_tempperp_fast'
temp_para_e = 'mms'+probe+'_des_temppara_fast'
temp_perp_e = 'mms'+probe+'_des_tempperp_fast'
number_density_i = 'mms'+probe+'_dis_numberdensity_fast'
number_density_e = 'mms'+probe+'_des_numberdensity_fast'
b_field = 'mms'+probe+'_fgm_b_gsm_srvy_l2'
b_magnitude = 'mms'+probe+'_fgm_b_gsm_srvy_l2_mag'

## Split the B-field magnitude into its own variable

In [6]:
times, data = get_data(b_field)
store_data(b_magnitude, data={'x': times, 'y': data[:, 3]})

## Interpolate the B-field data to the FPI time stamps

In [7]:
tinterpol(b_magnitude, number_density_i, newname=['b_mag_interpolated'])

tinterpol (linear) was applied to: b_mag_interpolated


## Extract the data from the pyTplot variables

In [8]:
btimes, bdata = get_data('b_mag_interpolated')

ipatimes, i_para_temp = get_data(temp_para_i)
ipetimes, i_perp_temp = get_data(temp_perp_i)
epatimes, e_para_temp = get_data(temp_para_e)
epetimes, e_perp_temp = get_data(temp_perp_e)

nitimes, i_n = get_data(number_density_i)
netimes, e_n = get_data(number_density_e)

## Calculate the magnetic pressure

note: 1.0e-8 comes from A-nT/m -> g/(s^2-cm)

In [9]:
Pmag = 1.0e-8*bdata**2/(2.0*mu0)

## Calculate the ion and electron temperatures

In [10]:
Te_total=(e_para_temp+2*e_perp_temp)/3.0
Ti_total=(i_para_temp+2*i_perp_temp)/3.0

## Calculate the plasma pressure

note: eV -> K conversion: 11604.505 K/eV

In [11]:
Pplasma = (i_n*11604.505*Ti_total+e_n*11604.505*Te_total)*Kb

## Calculate plasma beta

In [12]:
Beta = Pplasma/Pmag

## Store the data in pyTplot variables

In [13]:
store_data('plasma_beta', data={'x': btimes, 'y': Beta})
store_data('magnetic_pressure', data={'x': btimes, 'y': Pmag})
store_data('plasma_pressure', data={'x': btimes, 'y': Pplasma})

## Plot the results

In [14]:
tplot(['plasma_beta', 'plasma_pressure', 'magnetic_pressure'])